In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
import cv2
import os
from keras.utils import to_categorical
from keras.optimizers import SGD,RMSprop, adadelta, Adam

Using TensorFlow backend.


In [2]:
PATH = os.getcwd()
data_path = PATH + './mnist_png/training'
data_Dir_list = os.listdir(data_path)
print(data_Dir_list)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\OCR14PDBS3\\CNN Model Notebooks./mnist_png/training'

In [2]:
img_row = 28
img_col = 28
num_channel = 1
epoch = 20
num_classes = 10

In [4]:
img_data_list = []
images = []
i = 0

for dataset in data_Dir_list:
    img_list = os.listdir(data_path + '/' + dataset)
    for img in img_list:
        i = i+1
        ip_img = cv2.imread(data_path + '/' + dataset + '/' + img)
        ip_img = cv2.cvtColor(ip_img, cv2.COLOR_BGR2GRAY)
        ip_img = cv2.resize(ip_img, (img_row,img_col))
        img_data_list.append(ip_img)
    images.append(i)

print(images)

[6903, 14780, 21770, 28911, 35735, 42048, 48924, 56217, 63042, 70000]


In [5]:
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
img_data.shape

(70000, 28, 28)

In [6]:
#Expand dimesnsions
img_data= np.expand_dims(img_data, axis=4)
img_data.shape

C:\Users\Praneet\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  from ipykernel import kernelapp as app


(70000, 28, 28, 1)

In [7]:
#Preprocessing
n_samples = img_data.shape[0]
labels = np.ones((n_samples,), dtype='int64')

label_dictionary = {0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9'}

labels[:images[0]] = 0
labels[images[0]:images[1]] = 1
labels[images[1]:images[2]] = 2
labels[images[2]:images[3]] = 3
labels[images[3]:images[4]] = 4
labels[images[4]:images[5]] = 5
labels[images[5]:images[6]] = 6
labels[images[6]:images[7]] = 7
labels[images[7]:images[8]] = 8
labels[images[8]:images[9]] = 9

#One-Hot encoded format
Y = to_categorical(labels, num_classes)

In [8]:
Y

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [5]:
#Shuffle data
from sklearn.utils import shuffle
x, y = shuffle(img_data, Y, random_state=2)
#Split Dataset
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=2)

NameError: name 'img_data' is not defined

Now we will build our model

In [3]:
tf.reset_default_graph()

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, (2,2), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 5, 128)         32896     
__________

In [10]:
#Callbacks Tensorboard
from keras.callbacks import TensorBoard
tbcallback = TensorBoard(log_dir='./Graph/', histogram_freq=1, write_graph=True, write_images=True)

In [11]:
model.fit(X_train, Y_train, batch_size=32, epochs=epoch, validation_data=(X_test, Y_test), callbacks=[tbcallback], verbose=1)

Train on 52500 samples, validate on 17500 samples
Epoch 1/20
52500/52500 [==============================] - 95s 2ms/step - loss: 0.4471 - acc: 0.8553 - val_loss: 0.0736 - val_acc: 0.9779
Epoch 2/20
52500/52500 [==============================] - 98s 2ms/step - loss: 0.1502 - acc: 0.9585 - val_loss: 0.0609 - val_acc: 0.9819
Epoch 3/20
52500/52500 [==============================] - 107s 2ms/step - loss: 0.1205 - acc: 0.9669 - val_loss: 0.0495 - val_acc: 0.9853
Epoch 4/20
52500/52500 [==============================] - 95s 2ms/step - loss: 0.1006 - acc: 0.9723 - val_loss: 0.0444 - val_acc: 0.9863
Epoch 5/20
52500/52500 [==============================] - 94s 2ms/step - loss: 0.0942 - acc: 0.9735 - val_loss: 0.0415 - val_acc: 0.9877
Epoch 6/20
52500/52500 [==============================] - 94s 2ms/step - loss: 0.0866 - acc: 0.9764 - val_loss: 0.0396 - val_acc: 0.9890
Epoch 7/20
52500/52500 [==============================] - 95s 2ms/step - loss: 0.0810 - acc: 0.9774 - val_loss: 0.0431 - val_ac

In [4]:
model.load_weights('./models/weights.h5')

In [4]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'X_test' is not defined

In [13]:
target_dir = './newmodels/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./newmodels/model.h5')
model.save_weights('./newmodels/weights.h5')

In [8]:
import numpy as np
from keras.preprocessing import image
#test_image = image.load_img('43.png', target_size=(28,28))
ip_img = cv2.imread('54.png')
ip_img = cv2.cvtColor(ip_img, cv2.COLOR_BGR2GRAY)
ip_img = cv2.resize(ip_img, (28,28))
test_image = image.img_to_array(ip_img)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)[0]

print(result)
number = 0
for i in result[0]:
    if i == 1:
        result = number
    number += 1
print("The Number is = %d" % result )

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


TypeError: 'numpy.float32' object is not iterable

# Predicting Text from an Image

In [8]:
import os, PythonMagick
from PythonMagick import Image
from datetime import datetime
import PyPDF2
import numpy as np
import cv2
from imutils import contours
import imutils
import numpy as np
from keras.preprocessing import image

In [9]:
def x_cord_contour(cnts):
    
    if cv2.contourArea(cnts) > 5:
        M = cv2.moments(cnts)
        return (int(M['m10']/M['m00']))
    
def makeSquare(not_square):
    # Adds black pixels as padding
    
    BLACK = [0, 0, 0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square, (2 * width, 2 * height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        if (height > width):
            pad = (height - width)/2
            pad = int(pad)
            #doublesize = int(doublesize)
            doublesize_square = cv2.copyMakeBorder(doublesize, 0, 0, pad,\
                                                  pad, cv2.BORDER_CONSTANT, value = BLACK)
        else:
            pad = (width - height)/2
            pad = int(pad)
            doublesize_square = cv2.copyMakeBorder(doublesize, pad, pad, 0, 0,\
                                                  cv2.BORDER_CONSTANT, value = BLACK)
    doublesize_square_dim = doublesize_square.shape
    return doublesize_square

def resize_to_pixel(dimensions, image):
    
    buffer_pix = 4
    dimensions = dimensions - buffer_pix
    squared = image
    r = float(dimensions)/squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    widht_r = img_dim2[1]
    BLACK = [0, 0, 0]
    if (height_r > widht_r):
        resized = cv2.copyMakeBorder(resized, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value = BLACK)
    if (height_r < widht_r):
        resized = cv2.copyMakeBorder(resized, 1, 0, 0, 0, cv2.BORDER_CONSTANT, value = BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized, p, p, p, p, cv2.BORDER_CONSTANT, value = BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    return ReSizedImg

a = 0

In [10]:
pdf_dir = r"./Output_PDFs/"
img_dir = r"./Images/"
bg_colour = "#ffffff"

In [11]:
predicted_nums = []
for img in [img_file for img_file in os.listdir(img_dir) if img_file.endswith(".jpg")]:

    input_img = img_dir + img
    input_for_detection = cv2.imread(input_img)
    #detection(image)
    #cv2.imshow('Original', image)
    img_scaled = cv2.resize(input_for_detection, None, fx = 0.30, fy = 0.30)
    cv2.imshow('Resized',img_scaled)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # height, width = img_scaled.shape[:2]

    # start_row, start_col = height, width

    # end_row, end_col = int(height * .95), width

    # cropped = img_scaled[start_row:end_row, start_col:end_col]
    # cv2.imshow('Cropped',cropped)
    # #cv2.imshow('Translation', img_scaled)
    # cv2.waitKey()
    gray = cv2.cvtColor(img_scaled, cv2.COLOR_BGR2GRAY)

    #cv2.imshow('Gray', gray)
    #cv2.waitKey(0)

    # Blur image then find edges using Canny
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    #cv2.imshow('Blurred', blurred)
    #cv2.waitKey(0)

    #ret, regoi = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)
    #cv2.imshow('ROI',regoi)
    #cv2.waitKey(0)

    edged = cv2.Canny(blurred, 65, 150)
    cv2.imshow('Edges', edged)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #lines = cv2.HoughLinesP(edged, 1, np.pi / 180, 300, 5, 5)
    #print(lines.shape)

    #for x1, y1, x2, y2 in lines[0]:
       # cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 3)
        
    # cv2.imshow('lines', image)
    # cv2.waitKey(0)

    # Find contours
    ret, cnts, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort out contours left to right by using their x coordinates
    #contours = sorted(contours, key = x_cord_contour(contours), reverse = False)


    (cnts, boundingBoxes) = contours.sort_contours(cnts, method="left-to-right")
    clone = input_for_detection.copy()



    # Loop over the contours
    # for (i, c) in enumerate(contours)
    # for c in contours:
    ans =[]
    for c in cnts:
        # computing bounding box for rectangle
        (x, y, w, h) = cv2.boundingRect(c)
        
        if w >=10 and h >=25:
            roi = blurred[y:y +h, x:x + w]
            cv2.imshow("roi", roi)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            ret, roi = cv2.threshold(roi, 170, 255, cv2.THRESH_BINARY_INV)
            cv2.imshow("roi", roi)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            # roi = int(roi)
            squared = makeSquare(roi)
            final = resize_to_pixel(28, squared)
            l = str(a)
            cv2.imshow("final", final)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            predict_img = image.img_to_array(final)
            predict_img = np.expand_dims(predict_img, axis = 0)
            ans = model.predict(predict_img)
            #print(model.predict(predict_img))
            number = 0
            for i in ans[0]:
                if i == 1:
                    #print(ans)
                    ans = number
                    print(ans)
                number += 1
                
            predicted_nums.append(ans)
            #print("The Number is = %d" % result )
            #cv2.imshow("final", final)
            #cv2.imwrite("./Detected/img_"+l+".jpg",final)
            a= a + 1
            #cv2.waitKey(0)

5
2
2
8
5
2
2
7
2
8
0
1
5
5
3
2
8


In [21]:
print("The Number is = " + str(predicted_nums) )

The Number is = [5, 2, 2, 8, 5, 2, 2, 7, 2, 0, 5, 5, 3, 2, 8]
